In [1]:
import bs4
import httpx
import pandas as pd
import pandera as pa

import os
from io import BytesIO
from zipfile import ZipFile
from datetime import datetime, date

/tmp/ipykernel_42044/858984732.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
URL = "https://www.nhtsa.gov/nhtsa-datasets-and-apis"

columns = [
    "CMPLID",
    "ODINO",
    "MFR_NAME",
    "MAKETXT",
    "MODELTXT",
    "YEARTXT",
    "CRASH",
    "FAILDATE",
    "FIRE",
    "INJURED",
    "DEATHS",
    "COMPDESC",
    "CITY",
    "STATE",
    "VIN",
    "DATEA",
    "LDATE",
    "MILES",
    "OCCURENCES",
    "CDESCR",
    "CMPL_TYPE",
    "POLICE_RPT_YN",
    "PURCH_DT",
    "ORIG_OWER_YN",
    "ANTI_BRAKES_YN",
    "CRUISE_CONT_YN",
    "NUM_CYLS",
    "DRIVE_TRAIN",
    "FUEL_SYS",
    "FUEL_TYPE",
    "TRASN_TYPE",
    "VEH_SPEED",
    "DOT",
    "TIRE_SIZE",
    "LOC_OF_TIRE",
    "TIRE_FAIL_TYPE",
    "ORIG_EQUIP_YN",
    "MANUF_DT",
    "SEAT_TYPE",
    "RESTRAINT_TYPE",
    "DEALER_NAME",
    "DEALER_TEL",
    "DEALER_CITY",
    "DEALER_STATE",
    "DEALER_ZIP",
    "PROD_TYPE",
    "REPAIRED_YN",
    "MEDICAL_ATTN",
    "VEHICLES_TOWED_YN",
]

In [7]:
def create_client() -> httpx.Client:
    """
    Creates a common client for future http requests

    Returns:
        httpx.Client: client with ford proxies
    """
    ford_proxy = str(os.getenv("FORD_PROXY"))
    timeout_config = httpx.Timeout(10.0, connect=5.0)
    # proxy_mounts = {
    #     "http://": httpx.HTTPTransport(proxy=httpx.Proxy(ford_proxy)),
    #     "https://": httpx.HTTPTransport(proxy=httpx.Proxy(ford_proxy)),
    # }
    return httpx.Client(
        timeout=timeout_config,
        # mounts=proxy_mounts,
        verify=False,
    )

In [8]:
with create_client() as client:
    soup = bs4.BeautifulSoup(client.get(URL).text, "html.parser")

complaints = soup.select("#nhtsa_s3_listing > tbody")[3]
elements = [row for row in complaints.find_all("td")]

data_list = []


if len(elements) % 3 != 0:
    print("The list of elements does not contain complete data for each row.")
else:
    for i in range(0, len(elements), 3):
        url_elem = elements[i].find("a")
        size_elem = elements[i + 1]
        date_elem = elements[i + 2]

        data_dict = {
            "url": url_elem.get("href") if url_elem else None,
            "size": size_elem.text.strip() if size_elem else None,
            "updated_date": (
                datetime.strptime(date_elem.text.strip(" ET"), "%m/%d/%Y %I:%M:%S %p")
                if date_elem
                else None
            ),
        }
        data_list.append(data_dict)

In [9]:
def mount_dataset_from_content(info):
    if date.strftime(info["updated_date"], "%Y-%m-%d") >= str(
        os.getenv("LAST_COMPLAINT_WAVE_DATE")
    ):
        raise Exception("Datasets not updated")

    with create_client() as client:
        resp = client.get(info["url"], timeout=160).content

    with ZipFile(BytesIO(resp)) as myzip:
        with myzip.open("COMPLAINTS_RECEIVED_2020-2024.txt") as file:
            dataset = pd.read_csv(file, sep="\t", header=None, names=columns)

    return dataset

In [10]:
df = mount_dataset_from_content(info=data_list[0])
df.head()  # out of VPN: 23.7s

/tmp/ipykernel_42044/1331314576.py:12: DtypeWarning: Columns (28,30,33,38,41,44) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset = pd.read_csv(file, sep="\t", header=None, names=columns)


,CMPLID,ODINO,MFR_NAME,MAKETXT,MODELTXT,YEARTXT,CRASH,FAILDATE,FIRE,INJURED,...,RESTRAINT_TYPE,DEALER_NAME,DEALER_TEL,DEALER_CITY,DEALER_STATE,DEALER_ZIP,PROD_TYPE,REPAIRED_YN,MEDICAL_ATTN,VEHICLES_TOWED_YN
0,1633291,11292384,Honda (American Honda Motor Co.),HONDA,ACCORD,2018.0,N,20191221,N,0,...,NaN,NaN,NaN,NaN,NaN,NaN,V,NaN,N,N
1,1633292,11292384,Honda (American Honda Motor Co.),HONDA,ACCORD,2018.0,N,20191221,N,0,...,NaN,NaN,NaN,NaN,NaN,NaN,V,NaN,N,N
2,1633293,11292384,Honda (American Honda Motor Co.),HONDA,ACCORD,2018.0,N,20191221,N,0,...,NaN,NaN,NaN,NaN,NaN,NaN,V,NaN,N,N
3,1633294,11292385,Ford Motor Company,FORD,EXPLORER,2020.0,N,20191226,N,0,...,NaN,NaN,NaN,NaN,NaN,NaN,V,NaN,N,N
4,1633295,11292386,"General Motors, LLC",CHEVROLET,VOLT,2017.0,N,20190712,N,0,...,NaN,NaN,NaN,NaN,NaN,NaN,V,NaN,N,N


In [17]:
df["TIRE_SIZE"].value_counts()

TIRE_SIZE
ST225/75R15      17
* 33X12.5R22     12
285/60R20        10
* 205/75R14       8
255/50R20         6
                 ..
300/35/18         1
280/35/18         1
* 245/45R17       1
225/55/R17        1
* 275/75R22.5     1
Name: count, Length: 169, dtype: int64

In [19]:
df[
    (df["MFR_NAME"] == "Ford Motor Company")
    & (
        pd.to_datetime(df["DATEA"], format="%Y%m%d")
        > pd.Timestamp(str(os.getenv("LAST_COMPLAINT_WAVE_DATE")))
    )
    & (df["YEARTXT"].fillna(0).astype(int) > 2011)  # type: ignore
]

,CMPLID,ODINO,MFR_NAME,MAKETXT,MODELTXT,YEARTXT,CRASH,FAILDATE,FIRE,INJURED,...,RESTRAINT_TYPE,DEALER_NAME,DEALER_TEL,DEALER_CITY,DEALER_STATE,DEALER_ZIP,PROD_TYPE,REPAIRED_YN,MEDICAL_ATTN,VEHICLES_TOWED_YN
332548,1965917,11571573,Ford Motor Company,FORD,F-150,2019.0,N,20240211,N,0,...,NaN,NaN,NaN,NaN,NaN,NaN,V,NaN,N,N
332556,1965925,11571577,Ford Motor Company,FORD,EXPLORER,2013.0,N,20240213,N,0,...,NaN,NaN,NaN,NaN,NaN,NaN,V,NaN,N,N
332567,1965936,11571584,Ford Motor Company,FORD,ESCAPE,2021.0,N,20230701,N,0,...,NaN,NaN,NaN,NaN,NaN,NaN,V,NaN,N,N
332597,1965966,11571603,Ford Motor Company,FORD,ESCAPE,2020.0,N,20240211,N,0,...,NaN,NaN,NaN,NaN,NaN,NaN,V,NaN,N,N
332617,1965986,11571618,Ford Motor Company,LINCOLN,CORSAIR,2021.0,N,20240213,N,0,...,NaN,NaN,NaN,NaN,NaN,NaN,V,NaN,N,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
333470,1966839,11572229,Ford Motor Company,FORD,FUSION,2015.0,N,20240131,N,0,...,NaN,Heiser's Quick Motors,NaN,Milwaukee,WI,53223,V,NaN,N,Y
333472,1966841,11572231,Ford Motor Company,FORD,RANGER,2020.0,N,20240212,N,0,...,NaN,Kendall Ford,NaN,Murrysville,WA,98721,V,NaN,N,N
333473,1966842,11572232,Ford Motor Company,FORD,ECOSPORT,2018.0,N,20240120,N,0,...,NaN,NaN,NaN,NaN,NaN,NaN,V,NaN,N,N
333515,1966884,11572258,Ford Motor Company,FORD,BRONCO SPORT,2022.0,N,20240210,N,0,...,NaN,NaN,NaN,NaN,NaN,NaN,V,NaN,N,N
